In [190]:
# pip install pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName("covid").getOrCreate()

In [193]:
# ! wget = https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv

In [3]:
mobility_df = spark.read.csv('Global_Mobility_Report.csv',header=True, inferSchema=True)

In [4]:
spark = SparkSession.builder.config("spark.sql.repl.eagerEval.enabled", True).getOrCreate()

owid covid data set

In [196]:
# ! wget = https://raw.githubusercontent.com/owid/covid-19-data/cd2d25ec6b15b12a76c34b7f92f68c92b923cb75/public/data/owid-covid-data.csv

In [5]:
owid_covid_df = spark.read.csv('owid-covid-data.csv',header=True, inferSchema=True)

In [198]:
# mobility_df_aus = owid_covid_df.filter(col("location") == "Australia")

In [199]:
# mobility_df_aus

In [200]:
# mobility_df_uae = owid_covid_df.filter(col("location") == "United Arab Emirates")

In [201]:
# mobility_df_aus_agg = mobility_df_aus.groupBy("date","location").agg(
#     {
#      "retail_and_recreation_percent_change_from_baseline": "avg",
#      "grocery_and_pharmacy_percent_change_from_baseline": "avg",
#      "parks_percent_change_from_baseline": "avg",
#      "transit_stations_percent_change_from_baseline": "avg",
#      "workplaces_percent_change_from_baseline": "avg",
#      "residential_percent_change_from_baseline": "avg"}
# )

In [202]:
# mobility_df_uae_agg = mobility_df_uae.groupBy("date").agg(
#     {
#      "retail_and_recreation_percent_change_from_baseline": "avg",
#      "grocery_and_pharmacy_percent_change_from_baseline": "avg",
#      "parks_percent_change_from_baseline": "avg",
#      "transit_stations_percent_change_from_baseline": "avg",
#      "workplaces_percent_change_from_baseline": "avg",
#      "residential_percent_change_from_baseline": "avg"}
# )

In [7]:
mobility_df.columns

['country_region_code',
 'country_region',
 'sub_region_1',
 'sub_region_2',
 'metro_area',
 'iso_3166_2_code',
 'census_fips_code',
 'place_id',
 'date',
 'retail_and_recreation_percent_change_from_baseline',
 'grocery_and_pharmacy_percent_change_from_baseline',
 'parks_percent_change_from_baseline',
 'transit_stations_percent_change_from_baseline',
 'workplaces_percent_change_from_baseline',
 'residential_percent_change_from_baseline']

Joined the data set

In [8]:
mobility_df_agg = mobility_df.groupBy("date", "country_region").agg(
    {
        "retail_and_recreation_percent_change_from_baseline": "avg",
        "grocery_and_pharmacy_percent_change_from_baseline": "avg",
        "parks_percent_change_from_baseline": "avg",
        "transit_stations_percent_change_from_baseline": "avg",
        "workplaces_percent_change_from_baseline": "avg",
        "residential_percent_change_from_baseline": "avg"
    }
)


In [10]:
mobility_df_agg

date,country_region,avg(transit_stations_percent_change_from_baseline),avg(grocery_and_pharmacy_percent_change_from_baseline),avg(retail_and_recreation_percent_change_from_baseline),avg(workplaces_percent_change_from_baseline),avg(residential_percent_change_from_baseline),avg(parks_percent_change_from_baseline)
2021-07-13,United Arab Emirates,-29.333333333333332,22.75,5.125,-10.875,5.875,-25.857142857142858
2021-08-01,United Arab Emirates,-30.166666666666668,20.0,3.625,-11.0,5.875,-26.571428571428573
2021-09-10,United Arab Emirates,-29.5,20.625,4.125,-5.875,6.285714285714286,-31.857142857142858
2022-02-24,United Arab Emirates,7.666666666666667,28.625,14.125,17.25,8.0,3.4285714285714284
2022-09-17,United Arab Emirates,26.0,48.75,30.25,13.875,7.5,15.714285714285714
2020-09-03,Afghanistan,-2.0,NULL,NULL,-2.0,3.0,9.0
2021-07-29,Afghanistan,6.0,58.5,54.0,7.5,-3.0,36.5
2022-06-21,Afghanistan,15.0,172.0,81.0,31.5,7.0,115.5
2020-06-03,Antigua and Barbuda,-50.333333333333336,-15.5,-24.0,-32.25,18.0,-18.5
2020-10-11,Antigua and Barbuda,-46.0,-5.0,-10.0,-29.5,3.0,78.5


In [11]:
joined_data = mobility_df_agg.join(owid_covid_df, "date")
# joined_data.show()

In [12]:
joined_data

date,country_region,avg(transit_stations_percent_change_from_baseline),avg(grocery_and_pharmacy_percent_change_from_baseline),avg(retail_and_recreation_percent_change_from_baseline),avg(workplaces_percent_change_from_baseline),avg(residential_percent_change_from_baseline),avg(parks_percent_change_from_baseline),iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
2020-02-15,United Arab Emirates,2.8333333333333335,4.125,0.625,2.875,0.625,5.142857142857143,ARG,South America,Argentina,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2.0,0.0,NULL,NULL,people tested,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,11.11,4.5195777E7,16.177,31.9,11.198,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,NULL,5.0,76.67,0.825
2020-02-15,United Arab Emirates,2.8333333333333335,4.125,0.625,2.875,0.625,5.142857142857143,AUS,Oceania,Australia,15.0,0.0,0.0,NULL,NULL,0.0,0.588,0.0,0.0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,19.44,2.5499881E7,3.202,37.9,15.504,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NULL,3.84,83.44,0.939
2020-02-15,United Arab Emirates,2.8333333333333335,4.125,0.625,2.875,0.625,5.142857142857143,BEL,Europe,Belgium,1.0,0.0,0.0,NULL,NULL,0.0,0.086,0.0,0.0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,11.11,1.1589616E7,375.564,41.8,18.571,12.849,42658.576,0.2,114.898,4.29,25.1,31.4,NULL,5.64,81.63,0.916
2020-02-15,United Arab Emirates,2.8333333333333335,4.125,0.625,2.875,0.625,5.142857142857143,KHM,Asia,Cambodia,1.0,0.0,0.0,NULL,NULL,0.0,0.06,0.0,0.0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0.0,1.6718971E7,90.672,25.6,4.412,2.385,3645.07,NULL,270.892,4.0,2.0,33.7,66.229,0.8,69.82,0.582
2020-02-15,United Arab Emirates,2.8333333333333335,4.125,0.625,2.875,0.625,5.142857142857143,CAN,North America,Canada,7.0,0.0,0.0,NULL,NULL,0.0,0.185,0.0,0.0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2.78,3.7742157E7,4.037,41.4,16.984,10.797,44017.591,0.5,105.599,7.37,12.0,16.6,NULL,2.5,82.43,0.926
2020-02-15,United Arab Emirates,2.8333333333333335,4.125,0.625,2.875,0.625,5.142857142857143,CHN,Asia,China,68413.0,2055.0,4514.143,1663.0,142.0,122.571,47.531,1.428,3.136,1.155,0.099,0.085,1.17,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,75.46,1.439323774E9,147.674,38.7,10.641,5.929,15308.712,0.7,261.899,9.74,1.9,48.4,NULL,4.34,76.91,0.752
2020-02-15,United Arab Emirates,2.8333333333333335,4.125,0.625,

In [13]:
# joined_data = joined_data.orderBy("date")

In [14]:
# joined_data.show(10, truncate=True)
# joined_data.show(n=25, truncate=False, vertical=True)

Australia

In [19]:
aus_mobility_data = joined_data.filter(col("location") == "Australia")
uae_mobility_data = joined_data.filter(col("location") == "United Arab Emirates")

In [16]:
# aus_mobility_data

In [17]:
# uae_mobility_data

In [21]:

import matplotlib.pyplot as plt
import pandas as pd

# Convert the aggregated data to a Pandas DataFrame
australia_mobility_pd = aus_mobility_data.toPandas()

# Set the 'date' column as a datetime object
australia_mobility_pd['date'] = pd.to_datetime(australia_mobility_pd['date'])
australia_mobility_pd = australia_mobility_pd.orderBy('date')

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Plot each mobility category
ax.plot(australia_mobility_pd['date'], australia_mobility_pd['avg(retail_and_recreation_percent_change_from_baseline)'], label='Retail & Recreation')
ax.plot(australia_mobility_pd['date'], australia_mobility_pd['avg(grocery_and_pharmacy_percent_change_from_baseline)'], label='Grocery & Pharmacy')
ax.plot(australia_mobility_pd['date'], australia_mobility_pd['avg(parks_percent_change_from_baseline)'], label='Parks')
ax.plot(australia_mobility_pd['date'], australia_mobility_pd['avg(transit_stations_percent_change_from_baseline)'], label='Transit Stations')
ax.plot(australia_mobility_pd['date'], australia_mobility_pd['avg(workplaces_percent_change_from_baseline)'], label='Workplaces')
ax.plot(australia_mobility_pd['date'], australia_mobility_pd['avg(residential_percent_change_from_baseline)'], label='Residential')


# Set the x-axis label
ax.set_xlabel('Date')

# Set the y-axis label
ax.set_ylabel('Average Percent Change from Baseline')

# Set the title
ax.set_title('Mobility Trends in Australia')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add a legend
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()


AttributeError: 'DataFrame' object has no attribute 'orderBy'

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis for each subplot
fig, axs = plt.subplots(6, 1, figsize=(10, 12), sharex=True)

# Define the labels for each category
categories = ['retail_and_recreation', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces', 'residential']

# Loop through the categories and create a subplot for each
for i, category in enumerate(categories):
    axs[i].plot(australia_mobility_pd['date'], australia_mobility_pd[f'avg({category.lower()}_percent_change_from_baseline)'])
    axs[i].set_title(category)
    axs[i].grid(True)

# Add labels and adjust layout
plt.xlabel('Date')
plt.tight_layout()
plt.show()

UAE

In [ ]:
uae_mobility_data = joined_data.filter(col("location") == "United Arab Emirates")

import matplotlib.pyplot as plt
import pandas as pd

# Convert the aggregated data to a Pandas DataFrame
uae_mobility_pd = uae_mobility_data.toPandas()

# Set the 'date' column as a datetime object
uae_mobility_pd['date'] = pd.to_datetime(uae_mobility_pd['date'])

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Plot each mobility category
ax.plot(uae_mobility_pd['date'], uae_mobility_pd['avg(retail_and_recreation_percent_change_from_baseline)'], label='Retail & Recreation')
ax.plot(uae_mobility_pd['date'], uae_mobility_pd['avg(grocery_and_pharmacy_percent_change_from_baseline)'], label='Grocery & Pharmacy')
ax.plot(uae_mobility_pd['date'], uae_mobility_pd['avg(parks_percent_change_from_baseline)'], label='Parks')
ax.plot(uae_mobility_pd['date'], uae_mobility_pd['avg(transit_stations_percent_change_from_baseline)'], label='Transit Stations')
ax.plot(uae_mobility_pd['date'], uae_mobility_pd['avg(workplaces_percent_change_from_baseline)'], label='Workplaces')
ax.plot(uae_mobility_pd['date'], uae_mobility_pd['avg(residential_percent_change_from_baseline)'], label='Residential')


# Set the x-axis label
ax.set_xlabel('Date')

# Set the y-axis label
ax.set_ylabel('Average Percent Change from Baseline')

# Set the title
ax.set_title('Mobility Trends in UAE')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add a legend
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
uae_mobility_data

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis for each subplot
fig, axs = plt.subplots(6, 1, figsize=(10, 12), sharex=True)

# Define the labels for each category
categories = ['retail_and_recreation', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces', 'residential']

# Loop through the categories and create a subplot for each
for i, category in enumerate(categories):
    axs[i].plot(uae_mobility_pd['date'], uae_mobility_pd[f'avg({category.lower()}_percent_change_from_baseline)'])
    axs[i].set_title(category)
    axs[i].grid(True)

# Add labels and adjust layout
plt.xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis for each subplot
fig, axs = plt.subplots(6, 1, figsize=(10, 12), sharex=True)

# Define the labels for each category
categories = ['retail_and_recreation', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces', 'residential']

# Loop through the categories and create a subplot for each
for i, category in enumerate(categories):
    # Plot data for Australia
    axs[i].plot(australia_mobility_pd['date'], australia_mobility_pd[f'avg({category.lower()}_percent_change_from_baseline)'],
                label='Australia')
    
    # Plot data for UAE
    axs[i].plot(uae_mobility_pd['date'], uae_mobility_pd[f'avg({category.lower()}_percent_change_from_baseline)'],
                label='UAE')
    
    axs[i].set_title(category)
    axs[i].grid(True)
    axs[i].legend()  # Add a legend to distinguish Australia and UAE

# Add labels and adjust layout
plt.xlabel('Date')
plt.tight_layout()
plt.show()